In [ ]:
!pip install langchain_experimental
!pip install llamaapi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.2/279.2 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [ ]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_experimental.chat_models import Llama2Chat
from llamaapi import LlamaAPI
from langchain_experimental.llms import ChatLlamaAPI
from langchain.chains import create_tagging_chain
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


In [ ]:
llama = LlamaAPI("LL-DVLlvLm4vFiT8CtbRjgzsol1vGYIUcZU8GEfdSHRMPKVEiq34kc4xbHjXQuk0xhl")
model = ChatLlamaAPI(client=llama)

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
%cd '/content/drive/My Drive/ml-1m/'

Mounted at /content/drive/
/content/drive/My Drive/ml-1m


In [ ]:
template = ChatPromptTemplate.from_messages([
    ("system", "Please provide the groundtruth of the movie, including Movie ID, Title, Genres, Director, Lead Actor 1, Lead Actor 2, Lead Actor 3. These values should be separated by ::. If the movie has multiple genres, they should be connected with &. If you don't know, please use null instead of giving me something wrong."),
    ("human", "1,Toy Story (1995)"),
    ("ai", "1::Toy Story::Animation&Adventure&Comedy&Family::1995::John Lasseter::Tom Hanks::Tim Allen::Joan Cusack"),
    ("human", "2,Jumanji (1995)"),
    ("ai", "2::Jumanji::Adventure&Fantasy&Comedy&Family::1995::Joe Johnston::Robin Williams::Kirsten Dunst::Bonnie Hunt"),
    ("human", "3,Grumpier Old Men (1995)"),
    ("ai", "3::Grumpier Old Men::Comedy&Drama&Romance::1995::Howard Deutch::Jack Lemmon::Walter Matthau::Ann-Margret"),

    ("human", "{user_input}"),
])

chain = template | model

In [ ]:
import csv
import pandas as pd
import json

# 读取CSV文件并创建DataFrame对象
df = pd.read_csv('movies_info.csv')

# 选择从第五行开始的所有行
start_row_index = 3600  # 索引从0开始，第五行的索引为4
selected_rows = df.iloc[start_row_index:]

# print(selected_rows)
# 遍历选定的行
data=[]
for index, row in selected_rows.iterrows():
    # 访问"title"列的值
    id = row['MovieID']
    title = row['Title']
    # print(title)
    json_data = {
            'ID': id,
            'title': title,
        }
    # print(json_data)

    prompt_value = chain.invoke({
        "user_input": json_data})
    print(prompt_value.content)

        # 将数据添加到列表中
    data.append(prompt_value.content)


3669::Stay Tuned::Comedy::1992::Jim Henson::Jason Alexander::Michael McKean::Nancy Marchand
3670::Story of G.I. Joe, The::Action&Adventure&Drama::1945::William A. Wellman::Burgess Meredith::Lana Turner::Robert Mitchum[
3671::Blazing Saddles::Comedy&Western::1974::Mel Brooks::Cleavon Little::Gene Wilder::Slim Pickens[
3672::Benji::Drama&Family::1974::Joe Camp::Peter Breck::Nellie Bellflower::Gale Sondergaard
3673::Benji the Hunted::Adventure&Family::1987::Burt Kennedy::Alex Karras::Tommy Kirk::Danny Bonaduce
3674::For the Love of Benji::Drama&Family::1977::Bert I. Gordon::Peter Breck::Nancy McKeon::Tommy Kramer[
3675::White Christmas::Musical&Romance&Comedy::1954::Mark Sandrich::Bing Crosby::Danny Kaye::Rosemary Clooney[
3676::Eraserhead::Drama&Horror&Mystery::1977::David Lynch::Jack Nance::Laura Dern::Isabella Rossellini[
3677::Baraka::Documentary::1992::Ron Fricke::null::null::null
3678::The Man with the Golden Arm::Drama&Film Noir::1955::Otto Preminger::Frank Sinatra::Kim Novak::Elea

In [ ]:
with open('movies_3672.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    header = ['Movie ID','Movie Title', 'Movie Genre', 'Release Year', 'Director', 'Lead Actor 1', 'Lead Actor 2', 'Lead Actor 3']
    writer.writerow(header)
    for row in data:
        # 将字符串按逗号分隔成列表
        row_data = row.split('::')
        writer.writerow(row_data)